In [ ]:
from pathlib import Path
import xarray as xr
import numpy as np
import geopandas as gpd
import stmtools
import dask

## Data loading

In [ ]:
path_stm = Path('./full-pixel_psi_amsterdam_tsx_asc_t116_v4_sparse.csv')
stmat = stmtools.from_csv(path_stm, blocksize=10e6)
stmat = stmat.chunk({"space": 20000, "time": -1}) # Chunk 10000 space, no chunk in time
stmat

In [ ]:
dask.visualize(stmat['lat'], filename='lat.png')

In [ ]:
# Save to zarr
stmat.to_zarr('./full-pixel_psi_amsterdam_tsx_asc_t116_v4_sparse.zarr', mode='w')

## Exercise: Performance comparison between csv vs zarr (15 mins)

Read from the csv file and the Zarr file separately, access the `deformation` variable and load it into the memory using the `.compute()` function.

Use the magic command `%%time` to measure the time, compare the performance between the two file formats.

In [ ]:
%%timeit
path_csv = Path('./full-pixel_psi_amsterdam_tsx_asc_t116_v4_sparse.csv')
stmat1 = stmtools.from_csv(path_csv, blocksize=10e6)
stmat1 = stmat.chunk({"space": 20000, "time": -1})
defo_1 = stmat['deformation'].compute()

In [ ]:
%%timeit
path_zarr = Path('./full-pixel_psi_amsterdam_tsx_asc_t116_v4_sparse.zarr')
stmat2 = xr.open_zarr(path_zarr)
defo_2 = stmat2['deformation'].compute()

In [ ]:
path_zarr = Path('./full-pixel_psi_amsterdam_tsx_asc_t116_v4_sparse.zarr')
stmat = xr.open_zarr(path_zarr)

## Subset by polygon and Enrich the STM

In [ ]:
# Path to the BRP polygon of NL
path_polygon = Path('bag_light_AMS_WGS84.gpkg')

In [ ]:
# AoI boundary
min_lon = stmat['lon'].values.min()
max_lon = stmat['lon'].values.max()
min_lat = stmat['lat'].values.min()
max_lat = stmat['lat'].values.max()
bbox = (min_lon, min_lat, max_lon, max_lat)
bbox

In [ ]:
gdf = gpd.read_file(path_polygon, bbox=bbox)
gdf.plot()

In [ ]:
stmat_subset = stmat.stm.subset(method='polygon', polygon=path_polygon)
stmat_subset

In [ ]:
fields_to_query = ['bouwjaar']
stmat_enriched = stmat_subset.stm.enrich_from_polygon(path_polygon, fields_to_query)
stmat_enriched

In [ ]:
# scatter plot of the bouwjaar
data = stmat_enriched['bouwjaar'].compute()
data

In [ ]:
# Visualize results
from matplotlib import pyplot as plt
import matplotlib.cm as cm

colormap = cm.jet

plt.title("Construction year, PS")
plt.scatter(data['lon'], data['lat'], c=data, s=0.004, cmap=colormap)
plt.clim([1900, 2023])
plt.xlim([4.84, 4.96])
plt.ylim([52.34, 52.39])
plt.colorbar()

## Use `map_block` functions

In [ ]:
xr.map_blocks?

In [ ]:
# define the function to get deformation percentile per point
def get_percentile(stm, q=0.95):
    stm[f'pnt_defo_perc{q}'] = stm['deformation'].quantile(q, dim='time')
    stm = stm.drop_vars('quantile')
    return stm

In [ ]:
# make a small crop
stmat_subset = stmat.isel(space=slice(0, 1000))
stmat_subset

In [ ]:
# test function of the small crop
stmat_subset_perc = get_percentile(stmat_subset)
stmat_subset_perc

In [ ]:
# Make a template stmat, add the `'pnt_defo_perc0.95'` variable
stmat_template = stmat.copy()
stmat_template['pnt_defo_perc0.95'] = stmat['pnt_line']*0
stmat_template

In [ ]:
stmat_perc = stmat.map_blocks(get_percentile, kwargs={"q":0.95}, template=stmat_template)
stmat_perc

In [ ]:
stmat_perc['pnt_defo_perc0.95'].compute()